ModuleNotFoundError: No module named 'cd'

In [1]:
%load_ext autoreload
%autoreload 3
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import sys
import os
sys.path.append('../fit')
import cd


# check out how two models differ
import torch.optim as O
import torch.nn as nn

from torchtext import data
from torchtext import datasets
from os.path import join

In [2]:
%load_ext autoreload
%autoreload 4
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%reload_ext autoreload

In [4]:
from torchtext.data import TabularDataset

In [5]:
dataset_path = '../data'

In [6]:
inputs = data.Field(lower=True)
answers = data.Field(sequential=False, unk_token=None)
tv_datafields = [ ("text", inputs), ("answers", answers)]

In [7]:
train, dev, test = TabularDataset.splits(
                           path=dataset_path, # the root directory where the data lies
                           train='train_decoy_SST.csv', validation="dev_decoy_SST.csv", test = "test_decoy_SST.csv",
                           format='csv', 

                           skip_header=False, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                           fields=tv_datafields)

In [437]:
dataset_path = '../data'

In [438]:
train, dev, test = TabularDataset.splits(
                           path=dataset_path, # the root directory where the data lies
                           train='train_decoy_SST_' +str(100.0)+'.csv', validation="dev_decoy_SST.csv", test = "test_decoy_SST.csv",
                           format='csv', 
                           skip_header=False, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                           fields=tv_datafields)

In [439]:
len(train)

98794

In [440]:
vector_cache = '../data/.vector_cache/input_vectors.pt'
word_vectors = 'glove.6B.300d'

In [441]:
inputs.build_vocab(train, dev, test)
if word_vectors:
    if os.path.isfile(vector_cache):
        inputs.vocab.vectors = torch.load(vector_cache)
    else:
        inputs.vocab.load_vectors(word_vectors)
        os.makedirs(os.path.dirname(vector_cache), exist_ok=True)
        torch.save(inputs.vocab.vectors, vector_cache)
answers.build_vocab(train)

In [442]:
train_iter, dev_iter, test_iter = data.BucketIterator.splits(
    (train, dev, test), batch_size=3, sort_key=lambda x: len(x.text), sort = True, device=torch.device(0), shuffle = True)

In [ ]:
L

In [452]:
for batch_idx, batch in enumerate(train_iter):
    if batch_idx >20000:
        break

In [453]:
batch.text.shape

torch.Size([8, 3])

In [458]:
class_decoy = (inputs.vocab.stoi['text'], inputs.vocab.stoi['video'])

In [455]:
model_path = "../models/init_models"
model_list = os.listdir(model_path)
model = torch.load(join(model_path, model_list[0]), map_location=torch.device(0)).eval()

/home/lauri/.conda/envs/gpu_usage/lib/python3.6/site-packages/torch/serialization.py:435: SourceChangeWarning: source code of class 'model.LSTMSentiment' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [21]:
from cd import *

In [2]:
((batch.text ==class_decoy[0]) + (batch.text == class_decoy[1])).argmax(dim = 0)

NameError: name 'batch' is not defined

In [461]:
batch.text

tensor([[1474, 1440,   28],
        [  30,    5,  571],
        [1415, 1062,  508],
        [   2,  153,   38],
        [3463,    3,    4],
        [   7,   18,    3],
        [7849, 8735,  785],
        [2151,  590,  249]], device='cuda:0')

In [431]:
batch.text

tensor([[  29, 1471, 3136],
        [  79, 6595, 6381],
        [   5,   45,  523],
        [ 528,  512,    4],
        [  47,    9,  330],
        [   6, 2472, 2127],
        [1922,    4,    6],
        [2590, 7547,  206],
        [   4,    6,  157],
        [  17,    5,  853],
        [ 156,   65, 8506],
        [   5,  873,   57],
        [ 817, 1055,    5],
        [2274,   38,  112],
        [ 682,   41,   76],
        [ 250,  225,   21],
        [   2,    2,    2]], device='cuda:0')